# L1 Language Models, the Chat Format and Tokens

In [1]:
!pip install tiktoken

In [5]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [9]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    """Function to get the first response as the user"""
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output 
    )
    return response.choices[0].message["content"]

### Prompt the model and get a completion

In [10]:
response = get_completion("What is the capital of France?")

In [11]:
print(response)

The capital of France is Paris.


In [13]:
def get_completion_analyze(prompt, model="gpt-3.5-turbo"):
    """Function to get the first response as the user"""
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output 
    )
    return response

responses = get_completion_analyze(prompt="What is the capital of France?")
responses

<OpenAIObject chat.completion id=chatcmpl-9pfE3zsptFk7pFjAwveljzy2QvQEA at 0x75348a8ae270> JSON: {
  "id": "chatcmpl-9pfE3zsptFk7pFjAwveljzy2QvQEA",
  "object": "chat.completion",
  "created": 1722100227,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The capital of France is Paris."
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 14,
    "completion_tokens": 7,
    "total_tokens": 21
  },
  "system_fingerprint": null
}

### Tokens
- Why does it give the wrong answer? Because it does not predict the next word but the next token!
- Why it works? Because it tokenize and better see the individual letter.

In [14]:
response = get_completion("Take the letters in lollipop and reverse them")
print(response)

pilpolol


In [18]:
# The tricks is to add dashs to make it into tokens
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")

In [16]:
response

'p-o-p-i-l-l-o-l'

### Let's experience with the usage of LLMs

In [17]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    """Function that allows the randomness of the model's output and max number of tokens"""
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message["content"]

In [11]:
# Mock the role of the system, then ask the prompt!
messages =  [  
{'role':'system', 
 'content':"""You are an assistant who\
 responds in the style of Dr Seuss."""},    
{'role':'user', 
 'content':"""write me a very short poem\
 about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

Oh, the happy carrot in the ground so deep,
With its orange hue, it makes me leap!
In the garden, it grows so fine,
Bringing joy with each veggie vine.
So crunchy, so sweet, it's always a delight,
The happy carrot, shining bright!


In [19]:
messages = [
    {'role': 'system',
     'content': "You are a software engineer at BlackRock who are specialized in LLMs"},
    {'role': 'user',
     'content': "How many open sources LLMs are there?"}
]
response = get_completion_from_messages(messages, temperature=0)
print(response)

As of my last knowledge update, there are several open-source LLMs available for use. Some popular ones include OpenAI's GPT (Generative Pre-trained Transformer) models, Hugging Face's Transformers library, and EleutherAI's GPT-Neo. These models are widely used in natural language processing tasks and are continuously being developed and improved by the open-source community. It's always a good idea to check the latest repositories and resources to stay up-to-date on the available open-source LLMs.


In [21]:
# length
messages =  [  
{'role':'system',
 'content':'All your responses must be one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once there was a cheerful carrot named Charlie who brightened everyone's day with his radiant smile.


In [22]:
# combined
messages =  [  
{'role':'system',
 'content':"""You are an assistant who responds in the style of Dr Seuss. All your responses must be one sentence long."""},    
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

In a garden so bright, there lived a carrot so slight, who danced with delight in the sun's warm light.


### We can also output the summarized analysis of the output
- Note that the way that it's defined the tokens is various.

In [23]:
def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=500):
    """Function that generate results and provide the count tokens as well"""
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message["content"]
    
    # Get the number of token in the prompt, output, and total of both
    token_dict = {
        'prompt_tokens':response['usage']['prompt_tokens'],
        'completion_tokens':response['usage']['completion_tokens'],
        'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

In [24]:
messages = [
{'role':'system', 
 'content':"""You are an assistant who responds\
 in the style of Dr Seuss."""},    
{'role':'user',
 'content':"""write me a very short poem \ 
 about a happy carrot"""},  
] 
response, token_dict = get_completion_and_token_count(messages)

In [25]:
print(response)

In a garden so bright, a carrot did grow,
With a smile on its face, a happy little glow.
It danced in the sun, and wiggled its toes,
Oh, what a joyous veggie, how it happily grows!


In [26]:
print(token_dict)

{'prompt_tokens': 37, 'completion_tokens': 50, 'total_tokens': 87}
